In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 전처리 라이브러리
import pandas as pd
import numpy as np
import os
import datetime as dt

# 분석에 문제가 없는 경고 메시지 숨김
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 데이터 불러오기
os.chdir('/content/drive/MyDrive/롯데멤버스_경진대회/LPOINT_BIG_COMP/')
demo = pd.read_csv('LPOINT_BIG_COMP_01_DEMO.csv') # DEMO - 고객 데모 정보
pdde = pd.read_csv('LPOINT_BIG_COMP_02_PDDE.csv', parse_dates=['de_dt']) # 상품 구매 정보 - 유통사 상품 구매 내역
cop_u = pd.read_csv('LPOINT_BIG_COMP_03_COP_U.csv', parse_dates=['de_dt']) # 제휴사 이용 정보 - 제휴사 서비스 이용 내역
pd_clac = pd.read_csv('LPOINT_BIG_COMP_04_PD_CLAC.csv') # 상품 분류 정보 - 유통사 상품 카테고리 마스터
br = pd.read_csv('LPOINT_BIG_COMP_05_BR.csv') # 점포 정보 - 유통사/제휴사 점포 마스터
lpay = pd.read_csv('LPOINT_BIG_COMP_06_LPAY.csv') # lpay 이용 - 엘페이 결제 내역

In [ ]:
data = pdde[['cust','pd_c']].drop_duplicates(keep='last').reset_index(drop=True)
pd_df = pd_clac[['pd_c', 'clac_mcls_nm']]
df = pd.merge(data, pd_df, how='left')
# 번들 품목이 2개 이하 user 제외
cnt_df = df.groupby(['cust']).count().reset_index()
target = cnt_df['cust'][cnt_df['clac_mcls_nm']>2].tolist()
df = df.loc[df['cust'].isin(target)].reset_index(drop=True)
# int 형식 변환
df['cust_id'] = df['cust'].astype('category').cat.codes
df['pd_id'] = df['pd_c'].astype('category').cat.codes
df['mcls_id'] = df['clac_mcls_nm'].astype('category').cat.codes
df = df[['cust_id','mcls_id','pd_id']].drop_duplicates().sort_values('cust_id').reset_index(drop=True)

In [ ]:
# data split(train:validation:test=7:1:2)
train = df.groupby('cust_id')['mcls_id'].apply(lambda x: x[:int(.7*len(x))])
train = pd.DataFrame(train).reset_index().drop('level_1', axis=1)
tune = df.groupby('cust_id')['mcls_id'].apply(lambda x: x[int(.7*len(x)):int(.8*len(x))])
tune = pd.DataFrame(tune).reset_index().drop('level_1', axis=1)
test = df.groupby('cust_id')['mcls_id'].apply(lambda x: x[int(.8*len(x)):])
test = pd.DataFrame(test).reset_index().drop('level_1', axis=1)

In [ ]:
# bundle-item interaction
df_bundle_item = df[['mcls_id','pd_id']].drop_duplicates().sort_values('mcls_id')
# customer-item interaction
df_cust_item = df[['cust_id','pd_id']].drop_duplicates().sort_values('cust_id')
mcls_num = df_bundle_item[['mcls_id']].drop_duplicates().count()
pd_num = df_bundle_item[['pd_id']].drop_duplicates().count()
cust_num = df_cust_item[['cust_id']].drop_duplicates().count()
# data size
num = pd.DataFrame([[cust_num[0],mcls_num[0],pd_num[0]]], dtype=int)

In [ ]:
os.chdir('/content/drive/MyDrive/롯데멤버스_경진대회/0728 model/bgcn_data')

In [ ]:
# user_bundle_train.txt
train.to_csv('user_bundle_train.txt', header=False, index=False, sep = '\t')
# user_item.txt
df_cust_item.to_csv('user_item.txt', header=False, index=False, sep = '\t')
# bundle_item.txt
df_bundle_item.to_csv('bundle_item.txt', header=False, index=False, sep = '\t')
# data_size.txt
num.to_csv('Lpoint_data_size.txt', header=False, index=False, sep = '\t')
# user_bundle_tune.txt
tune.to_csv('user_bundle_tune.txt', header=False, index=False, sep = '\t')
# user_bundle_test.txt
test.to_csv('user_bundle_test.txt', header=False, index=False, sep = '\t')